In [1]:
import iris
import numpy as np
import xarray as xr
import calendar
import cartopy.util
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
# Read data
path_to_geos = Path('../../external_data/GEOS-Chem')
path_to_ukca = Path('../data')
geos = xr.open_dataset(path_to_geos / 'ocean_rono2_2013.nc')
ukca = iris.load_cube(str(path_to_ukca / 'xnvtc' / 'xnvtc_o3.nc'))
# ukca_srf = xr.open_dataset('/local/mwe14avu/UEA/PhD/models/ukca/data/ukca_coords_n96e_marcus_180705.nc')

In [ ]:
# 'GEOS-Chem: ', geos.data_vars, geos.coords
# 'UKCA: ', ukca.data_vars, ukca.coords
# GEOS-Chem: lon 2.5 lat 2 (at poles step is 1.5): (144, 91) 
# UKCA lon 3.75 lat 2.5 (96, 73)
# GEOS-Chem data doesn't have a land mask and doesn't have fill_values where land is, so need to need to ensure that 
# non-zero fluxes don't appear on land in UKCA and that there aren't any missing grid boxes.
# For that need to compare UKCA surface_altitude variable with GEOS-Chem fluxes.
# geos.MeNO3_emission.lon.values
# ukca.longitude.values

In [ ]:
# Delete unnecessary UKCA coordinates leaving only time, latitude and longitude
ukca.remove_coord('model_level_number')
ukca.remove_coord('forecast_reference_time')
ukca.remove_coord('forecast_period')
ukca.remove_coord('level_height')
ukca.remove_coord('sigma')

In [56]:
# Convert GEOS-Chem xarray dataset to iris cube
geos_meono2_emis = geos.MeNO3_emission.to_iris()
geos_etono2_emis = geos.EtNO3_emission.to_iris()
# Rename coordinates as iris expects
geos_meono2_emis.coord('Time, instantaneous or at start of averaging period').rename('time')
geos_meono2_emis.coord('Latitude').rename('latitude')
geos_meono2_emis.coord('Longitude').rename('longitude')
geos_etono2_emis.coord('Time, instantaneous or at start of averaging period').rename('time')
geos_etono2_emis.coord('Latitude').rename('latitude')
geos_etono2_emis.coord('Longitude').rename('longitude')
# Regrid GEOS-Chem data onto UKCA grid
ukca_grid = [('longitude', ukca.coord('longitude').points-180), ('latitude', ukca.coord('latitude').points)]
_ukca_meono2_emis = geos_meono2_emis.interpolate(ukca_grid, iris.analysis.Linear())
_ukca_etono2_emis = geos_etono2_emis.interpolate(ukca_grid, iris.analysis.Linear())
# Variables to convert molecules cm-2 s-1 to kg m-2 s-1
avogadro_number = iris.coords.AuxCoord(6.022*1e23, units='molec/mol', long_name='avogadro_number')
molar_mass_meono2 = iris.coords.AuxCoord(77.0394, units='g/mol', long_name='molar_mass_of_methyl_nitrate')
molar_mass_etono2 = iris.coords.AuxCoord(91.0660, units='g/mol', long_name='molar_mass_of_ethyl_nitrate')
# Convert to kg m-2 s-1
ukca_meono2_emis = _ukca_meono2_emis / avogadro_number * molar_mass_meono2 
ukca_etono2_emis = _ukca_etono2_emis / avogadro_number * molar_mass_etono2 
ukca_meono2_emis.convert_units('kg/m2/s')
ukca_etono2_emis.convert_units('kg/m2/s')
ukca_meono2_emis.rename('methyl nitrate oceanic emission')
ukca_etono2_emis.rename('ethyl nitrate oceanic emission')
ukca_meono2_emis

Methyl Nitrate Oceanic Emission (kg/m2/s),time,latitude,longitude
Shape,12,73,96
Dimension coordinates,,,
time,x,-,-
latitude,-,x,-
longitude,-,-,x


In [54]:
# for i in iris.std_names.STD_NAMES.keys():
#     if 'methyl_' in i:
#         print(i)

In [57]:
ukca_etono2_emis

Ethyl Nitrate Oceanic Emission (kg/m2/s),time,latitude,longitude
Shape,12,73,96
Dimension coordinates,,,
time,x,-,-
latitude,-,x,-
longitude,-,-,x


### Check global burdens

Fisher et al (2018): global RONO2 oceanic emissions for year 2013: MeONO2 is 157 Gg N a-1, EtONO2 is 27 Gg N a-1.

In [16]:
# Variables to convert to kg of N
avogadro_number_cb = iris.cube.Cube(6.022*1e23, units='molec/mol', long_name='avogadro_number')
molar_mass_meono2_cb = iris.cube.Cube(77.0394, units='g/mol', long_name='molar_mass_of_methyl_nitrate')
molar_mass_etono2_cb = iris.cube.Cube(91.0660, units='g/mol', long_name='molar_mass_of_ethyl_nitrate')
molar_mass_n_cb = iris.cube.Cube(14.0067, units='g/mol', long_name='molar_mass_of_atomic_nitrogen')

Start with GEOS-Chem:

In [17]:
# Guess coordinate bounds
for coord in ['longitude', 'latitude']:
    if not geos_meono2_emis.coord(coord).has_bounds():
        geos_meono2_emis.coord(coord).guess_bounds()
    if not geos_etono2_emis.coord(coord).has_bounds():
        geos_etono2_emis.coord(coord).guess_bounds()
# Calculate emission per year by weighting monthly values by month's length and adding values up
n_days_per_month = np.asarray([calendar.monthrange(2013, i)[1] for i in range(1, 13)]) # number of days in each month in year 2013
geos_month_weights = iris.util.broadcast_to_shape(n_days_per_month, geos_meono2_emis.shape, (0,)) # weight of each month's length
geos_meono2_emis_per_year = (geos_meono2_emis * 
                             iris.coords.AuxCoord(86400, units='s', long_name='seconds_per_day')).collapsed(['time'],
                                                                                                            iris.analysis.SUM,
                                                                                                            weights=geos_month_weights)
geos_etono2_emis_per_year = (geos_etono2_emis * 
                             iris.coords.AuxCoord(86400, units='s', long_name='seconds_per_day')).collapsed(['time'],
                                                                                                            iris.analysis.SUM,
                                                                                                            weights=geos_month_weights)
# Add up all grid cells accounting for their varying area sizes
geos_norm_area_weights = iris.analysis.cartography.area_weights(geos_meono2_emis_per_year, normalize=True)
geos_meono2_emis_per_year_area_sum = geos_meono2_emis_per_year.collapsed(['longitude', 'latitude'], 
                                                                         iris.analysis.SUM,
                                                                         weights=geos_norm_area_weights)
geos_etono2_emis_per_year_area_sum = geos_etono2_emis_per_year.collapsed(['longitude', 'latitude'], 
                                                                         iris.analysis.SUM,
                                                                         weights=geos_norm_area_weights)

In [18]:
# Calculate Earth's area
area_weights = iris.analysis.cartography.area_weights(geos_meono2_emis_per_year, normalize=False)
earth_area = iris.cube.Cube(area_weights.sum(), units='m2')

In [19]:
# Calculate GEOS-Chem global burden
geos_meono2_emis_burden = geos_meono2_emis_per_year_area_sum * earth_area / avogadro_number_cb * molar_mass_n_cb
geos_etono2_emis_burden = geos_etono2_emis_per_year_area_sum * earth_area / avogadro_number_cb * molar_mass_n_cb
geos_meono2_emis_burden.convert_units('Gg')
geos_etono2_emis_burden.convert_units('Gg')
print(geos_meono2_emis_burden.data, 'Gg')
print(geos_etono2_emis_burden.data, 'Gg')

157.12963628790993 Gg
26.948046324634973 Gg


Same for UKCA:

In [21]:
# Guess coordinate bounds
for coord in ['longitude', 'latitude']:
    if not ukca_meono2_emis.coord(coord).has_bounds():
        ukca_meono2_emis.coord(coord).guess_bounds()
    if not ukca_etono2_emis.coord(coord).has_bounds():
        ukca_etono2_emis.coord(coord).guess_bounds()
# Calculate emission per year by weighting monthly values by month's length and adding values up
ukca_month_weights = iris.util.broadcast_to_shape(n_days_per_month, ukca_meono2_emis.shape, (0,))
ukca_meono2_emis_per_year = (ukca_meono2_emis * 
                             iris.coords.AuxCoord(86400, units='s', long_name='seconds_per_day')).collapsed(['time'],
                                                                                                            iris.analysis.SUM,
                                                                                                            weights=ukca_month_weights)
ukca_etono2_emis_per_year = (ukca_etono2_emis * 
                             iris.coords.AuxCoord(86400, units='s', long_name='seconds_per_day')).collapsed(['time'],
                                                                                                            iris.analysis.SUM,
                                                                                                            weights=ukca_month_weights)
# Add up all grid cells accounting for their varying area sizes
ukca_norm_area_weights = iris.analysis.cartography.area_weights(ukca_meono2_emis_per_year, normalize=True)
ukca_meono2_emis_per_year_area_sum = ukca_meono2_emis_per_year.collapsed(['longitude', 'latitude'],
                                                                         iris.analysis.SUM, 
                                                                         weights=ukca_norm_area_weights)
ukca_etono2_emis_per_year_area_sum = ukca_etono2_emis_per_year.collapsed(['longitude', 'latitude'],
                                                                         iris.analysis.SUM, 
                                                                         weights=ukca_norm_area_weights)
# Calculate UKCA global burden
ukca_meono2_emis_burden = ukca_meono2_emis_per_year_area_sum * earth_area * avogadro_number_cb / molar_mass_meono2_cb * molar_mass_n_cb
ukca_etono2_emis_burden = ukca_etono2_emis_per_year_area_sum * earth_area * avogadro_number_cb / molar_mass_etono2_cb * molar_mass_n_cb
ukca_meono2_emis_burden.convert_units('Gg')
ukca_etono2_emis_burden.convert_units('Gg')
print(ukca_meono2_emis_burden.data, 'Gg')
print(ukca_etono2_emis_burden.data, 'Gg')

155.65567960242137 Gg
26.69187679297173 Gg


### Create perpetual year 2013 emissions

In [87]:
from datetime import datetime
import dateutil.parser

In [59]:
perpetual_meono2 = np.repeat(ukca_meono2_emis.data, 10, axis=0)
perpetual_etono2 = np.repeat(ukca_etono2_emis.data, 10, axis=0)

In [83]:
ukca.coord('time').units

Unit('hours since 1970-01-01 00:00:00', calendar='360_day')

In [84]:
time = iris.coords.DimCoord(np.arange(1,121,1), standard_name='time', units=ukca.coord('time').units)

In [85]:
iris.cube.Cube(perpetual_meono2, long_name='methyl_nitrate_oceanic_emission', units='kg/m2/s',
               dim_coords_and_dims=[(time, 0), (ukca.coord('latitude'), 1), (ukca.coord('longitude'), 2)])

Methyl Nitrate Oceanic Emission (kg/m2/s),time,latitude,longitude
Shape,120,73,96
Dimension coordinates,,,
time,x,-,-
latitude,-,x,-
longitude,-,-,x


In [93]:
ukca_hours_since_1970_01_01 = dateutil.parser.parse(str(ukca.coord('time').units)[11:])

In [94]:
ukca_hours_since_1970_01_01

datetime.datetime(1970, 1, 1, 0, 0)